## Rhynie Chert Metanetwork --> SLNs ##
### Adapted from "Analytical approaches to networks, trophic structure, and ancient food webs" NAPC 2024 food webs workshop ###

The following metanetwork consists of two files. The first lists and names the trophic guilds, along with the taxon richness of each guild and an assumption of its trophic habit (e.g. "producer"). Note that all guilds have been labelled as type "producer", which is obviously an error. This is intentional and will be corrected in a later section. The second file is the metanetwork adjacency matrix, which is a binary $\vert U\vert\times \vert U\vert$ matrix, where $\vert U\vert$ is the total number of guilds in the metanetwork, $U$. The entries in this matrix are 0 or 1. If guild $G_i$ contains species hypothesized to prey on some or all those in guild $G_j$, then the $ij^{th}$ entry is 1, and zero otherwise.

In [90]:
# load necessary Julia libraries
# these must be installed via the Julia repl or terminal environment. Do so with the following commands
# using Pkg
# Pkg.add("CSV")
using CSV,DelimitedFiles,DataFrames,Random,Distributions,LinearAlgebra,PoissonRandom,Graphs,FilePathsBase

In [91]:
# also load these library functions written for the SLN family of code
include("./SLN_maker.jl")
include("./r_no_prey.jl")

r_no_prey (generic function with 1 method)

In [92]:
γ = 3

n_reps = 1000
label = "lumped" #can fill in with a number or string to label multiple runs
analysis_type = "complete" #complete, terr, or aqu

"complete"

### File input ###
Read the metanetwork and adjacency matrix files and print the metanetwork as a simple file check.

In [93]:
P = CSV.read("Rhynie_lumped_data/guilds_lumped.csv",DataFrame)
A = readdlm("Rhynie_lumped_data/guild_matrix_lumped.csv", ',', Int8)
print(P)

20×8 DataFrame
 Row │ guild_no  guild_name                         major_taxa  G      sp_tp     resource_guilds                   terr   aqu   
     │ Int64     String                             Missing     Int64  String15  String?                           Int64  Int64 
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │        1  terrestrial detritus                  missing      1  producer  missing                               1      0
   2 │        2  land plants                           missing      8  producer  missing                               1      0
   3 │        3  heterotrophic microbes                missing      1  consumer  1,3                                   1      0
   4 │        4  terrestrial fungi                     missing      1  consumer  1,2,4                                 1      0
   5 │        5  terrestrial microbivores              missing      1  consumer  3,4,1

Note that the first several guilds are "producer" guilds, including detritus.

Also note that all the guilds are listed as trophic type "producer". We correct this in the following.

In [94]:
# check trophic positions and modify if necessary
# transpose A
A_t = transpose(A)
for i = 1:size(P,1)
    vA = sum(A[i,:],dims=1)
    vA_t = sum(A_t[i,:],dims=1)
    if vA[1]==0
        P[i,:sp_tp] = "producer"
    end
    if vA[1]!=0
        if vA_t[1]!=0
            P[i,:sp_tp] = "consumer"
        end
        if vA_t[1]==0
            P[i,:sp_tp] = "apex"
        end
    end
end
print(P)

20×8 DataFrame
 Row │ guild_no  guild_name                         major_taxa  G      sp_tp     resource_guilds                   terr   aqu   
     │ Int64     String                             Missing     Int64  String15  String?                           Int64  Int64 
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │        1  terrestrial detritus                  missing      1  producer  missing                               1      0
   2 │        2  land plants                           missing      8  producer  missing                               1      0
   3 │        3  heterotrophic microbes                missing      1  consumer  1,3                                   1      0
   4 │        4  terrestrial fungi                     missing      1  consumer  1,2,4                                 1      0
   5 │        5  terrestrial microbivores              missing      1  consumer  3,4,1

### Calculate summary stats and separate aquatic and terrestrial webs ###
Create subsets of the main web separating aquatic and terrestrial components

In [95]:
# calculate metanetwork diversity
# no. of guilds
no_guilds = size(P,1)
# calculate number of species
S = sum(P[:,:G])
no_species = S[1]

# if terrestrial- or aquatic-only analysis, replace metrics with subsetted versions
if analysis_type == "terr_only" || analysis_type == "terr"
    guilds_terr = findall(==(1),P[:,7])
    P = P[guilds_terr,:]
    A = A[guilds_terr,guilds_terr]
    no_guilds = size(P, 1)
    S = sum(P[:,:G])
    no_species = S[1]
    println("\nSelected terrestrial-only web containing $no_guilds guilds and $no_species species.")
elseif analysis_type == "aqu_only" || analysis_type == "aqu"
    guilds_aqu = findall(==(1),P[:,8])
    P = P[guilds_aqu,:]
    A_t = A_t[guilds_aqu,guilds_aqu]
    no_guilds = size(P, 1)
    S = sum(P[:,:G])
    no_species = S[1]
    println("\nSelected aquatic-only web containing $no_guilds guilds and $no_species species.")
else
    println("\nSelected complete web containing $no_guilds guilds and $no_species species.")
end


Selected complete web containing 20 guilds and 37 species.


## SLN construction ##

The next series of cells estimate a series of SLNs (species-level networks or food webs) from the metanetwork. Some of the code consists of essential algorithmic steps, but the general procedure is as follows:
1. Calculate the maximum number of potential prey species or nodes for each consumer. This is the sum richness of all the guilds that are linked as resources or prey, in the metanetwork, to a guild $G_i$. Designate this sum as $M_i = \sum_j^{\vert U\vert}a_{ij}\vert G_j\vert$.
2. Generate the specific number of prey species or resource nodes of a species in guild $G_i$. This depends on a stochastic draw from an in-degree distribution, which can be a hypothetical, model, or empirically determined distribution. Here we use a mixed exponential-power law distribution. Use of this distribution requires a parameter, $\gamma$, describing the thickness of the distribution's tail. The stochastic draw is done using the function "r_no_prey.jl", but it is quite easy to substitute another function using Julia's Distributions package. See https://juliastats.org/Distributions.jl/stable/starting/
3. When the number of prey, or in-degree of a species has been so determined, the actual prey species are then determined by assigning the number of links randomly to species within the prey guilds.

In [96]:
# construct guild x species array
meta_SLN = SLN_maker(A,P,no_guilds,no_species);

# calculate no. of prey species per guild
P[:,:no_prey] .= 0.0
P[:,:no_preds] .= 0.0
for i = 1:no_guilds
	  for j = 1:no_guilds
	      if A[i,j] == 1
	          P[i,:no_prey] = P[i,:no_prey] + P[j,:G]
	      end
        if A[j,i] == 1
	          P[i,:no_preds] = P[i,:no_preds] + P[j,:G]
	      end
	  end
end
print(P)

20×10 DataFrame
 Row │ guild_no  guild_name                         major_taxa  G      sp_tp     resource_guilds                   terr   aqu    no_prey  no_preds 
     │ Int64     String                             Missing     Int64  String15  String?                           Int64  Int64  Float64  Float64  
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │        1  terrestrial detritus                  missing      1  producer  missing                               1      0      0.0       4.0
   2 │        2  land plants                           missing      8  producer  missing                               1      0      0.0       4.0
   3 │        3  heterotrophic microbes                missing      1  consumer  1,3                                   1      0      2.0       7.0
   4 │        4  terrestrial fungi                     missing      1  consumer  1,2,4             

set $\gamma$, the power law parameter.

Specify a number of replicate SLNs to generate.

In [ ]:
# create directory to store analyses if it doesn't already exist
dir_path = "SLNs/Rhynie_$(label)"
mkpath(dir_path)

## beginning of loop that generates chosen # of SLNs (based on n_reps) and outputs interaction matrices + species guild assignments
for rep in 1:n_reps
    # make empty dataframe for species data
    species = DataFrame(sp_name = Int64[], guild = String[], guild_no = Int64[], guild_richness = Int64[], guild_no_prey = Int64[], guild_no_preds = Int64[], terr = Int64[], aqu = Int64[], sp_no_prey = Int64[], sp_no_preds = Int64[])

    # push guild data
    begin
        tally1 = [1]
        for i = 1:no_guilds
            guild_richness = P[i,:G]
            for j = 1:guild_richness
                push!(species, [tally1[1], P[i,:guild_name], i, P[i,:G], P[i,:no_prey], P[i,:no_preds], P[i,:terr], P[i,:aqu],0,0])
                tally1[1] = tally1[1] + 1
            end
        end
    end
        
    # initial species no. of prey; uses in-degree distribution
    for i = 1:size(species,1)
        species[i,:sp_no_prey] = r_no_prey(species[i,:guild_no_prey],γ)
    end
        
    # select species-specific prey and generate species A matrix
    sp_A = zeros(Int64,no_species,no_species)
    for i = 1:no_species
        #vector of species prey indices
        guild_prey = Int64[]
        for j = 1:no_species
            if meta_SLN[species[i,:guild_no],j] == 1
                push!(guild_prey,j)
            end
        end
        #randomize prey vector
        shuffle!(guild_prey)
        #pick species prey
        for k = 1:species[i,:sp_no_prey]
            sp_A[i,guild_prey[k]] = 1
        end
    end

    #calculate no. preds, or out-degree
    for i = 1:no_species
        out_degree = 0
        for j = 1:no_species
            if sp_A[j,i] == 1
                out_degree += 1
            end
        end
        species[i,:sp_no_preds] = out_degree
    end

    writedlm("$(dir_path)/matrix_$rep.csv", sp_A, ',')

    #save species info (the only thing we need is the guild IDs, the other info could be recalculated from the adjacency matrix)
    CSV.write("$(dir_path)/speciesinfo$rep.csv", species)
end
